In [1]:
import pandas as pd
import sys,os,copy,pdb,importlib
sys.path.append('../classes')
sys.path.append('../analysis')
import numpy as np
np.set_printoptions(precision=3,suppress=True)
import matplotlib.pyplot as plt

import tournament_new as tn
import create_database as cd
importlib.reload(tn)

import beck.beck_game
importlib.reload(beck.beck_game)
from beck.beck_game import BeckGame as Game
from beck.beck_players import HumanBeckPlayer
import arena
importlib.reload(arena)
from arena import Arena

import mcts
from bfts import BFTS

import cog_related
importlib.reload(cog_related)
from cog_related import cog_value_net as cvn


importlib.reload(tn)
game = Game(4,9,4)
all_p = pd.read_pickle(cd.DATABASE_LOC)

from pickle import Pickler, Unpickler
import pickle

# res = tn.merge_res_to_base()
tournament_res = pd.read_pickle('/scratch/zz737/fiar/tournaments/ai_all_player_round_robin_base.pkl')

from utils import *
mctsargs = dotdict({
        'numMCTSSims': 100,
        'cpuct': 2,
        })
import supervised_learning as sl
from importlib import reload
reload(sl)

<module 'supervised_learning' from '../classes/supervised_learning.py'>

In [97]:
checkpoint_dir = '/scratch/zz737/fiar/sl/resnet/res15_color_Ex_tournament_6_checkpoints_mcts100_cpuct2_id_1_checkpoint_55'
file = 'final'

checkpoint_dir = '/scratch/zz737/fiar/sl/resnet/res15_color_Ex_tournament_8_checkpoints_mcts100_cpuct2_id_res3-0_checkpoint_61'
file = 'final'

args = pickle.load(open(os.path.join(checkpoint_dir,'args.p'),'rb'))
on_c = sl.OthelloNNet_resnet(game, args)
nnet_c = sl.NNetWrapper_color(game,nnet=on_c,args=args)
nnet_c.load_checkpoint(checkpoint_dir, file+'.pth.tar')

In [100]:
# ex_dir = f'/scratch/zz737/fiar/tournaments/tournament_6/checkpoints_mcts100_cpuct2_id_1'
# ex_dir = f'/scratch/zz737/fiar/tournaments/tournament_9/checkpoints_mcts100_cpuct2_id_res9-0'
ex_dir = f'/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-0'
# ex_dir = f'/scratch/zz737/fiar/tournaments/tournament_9/checkpoints_mcts100_cpuct2_id_res3-0'
# ex_fn = f'checkpoint_28.pth.tar'
ex_fn = f'checkpoint_61.pth.tar'

load_folder_file = (ex_dir, ex_fn)

modelFile = os.path.join(load_folder_file[0], load_folder_file[1])
examplesFile = modelFile + ".examples"
with open(examplesFile, "rb") as f:
    trainExamplesHistory = Unpickler(f).load()
trainExamples = []
for e in trainExamplesHistory:
    trainExamples.extend(e)
from random import shuffle
shuffle(trainExamples)
trainExamples = [(*x,int(x[0].sum()==0)) for x in trainExamples] # black: sum to 0, 1; white: not sum to 0, 0

In [14]:
trainExamples

[(array([[ 0,  0,  1,  0,  1, -1,  0,  0,  0],
         [ 0,  0, -1, -1,  0,  1,  0,  0,  0],
         [ 0,  0,  1,  0, -1, -1,  1,  0,  0],
         [ 0, -1,  1,  1,  1, -1,  0, -1,  0]]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]),
  0.0001,
  1),
 (array([[ 0,  0,  0,  0,  0,  1,  0,  0,  0],
         [ 0,  0,  0, -1,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0, -1,  1, -1,  0,  0,  0]]),
  array([0.   , 0.   , 0.008, 0.008, 0.   , 0.   , 0.   , 0.008, 0.008,
         0.   , 0.   , 0.   , 0.   , 0.008, 0.008, 0.008, 0.   , 0.   ,
         0.   , 0.   , 0.008, 0.916, 0.   , 0.008, 0.   , 0.008, 0.   ,
         0.   , 0.   , 0.008, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]),
  -0.0001,
  0),
 (array([[ 1,  0,  0, -1,  1, -1,  1,  0,  0],
         [ 1, -1, -1, -1,  1, -1, -1, -1,  1],
         [ 0,  1, -1,  1, -1, 

In [102]:
examples = trainExamples[:2000]
input_boards, target_pis, target_vs, colors = list(zip(*examples))
input_boards = np.asarray(input_boards)
target_pis = np.asarray(target_pis)
target_vs = np.asarray(target_vs)
color_input = np.full((4,9,len(colors)),colors).swapaxes(0,-1).swapaxes(1,2) # turn color_batch into b x 4 x 9

In [103]:
nnet_c.nnet.model.evaluate([input_boards,color_input],[target_pis,target_vs],batch_size=100)

20/20 [==============================] - 10s 513ms/step - loss: 0.9846 - pi_loss: 0.7219 - v_loss: 0.1142


[0.9845664501190186, 0.7218695282936096, 0.11415548622608185]

In [104]:
nnet_c.nnet.model.evaluate([input_boards,np.abs(1-color_input)],[target_pis,target_vs],batch_size=100)

20/20 [==============================] - 10s 516ms/step - loss: 1.5482 - pi_loss: 1.1285 - v_loss: 0.2712


[1.5482182502746582, 1.128466010093689, 0.27121052145957947]

<module 'mcts' from '../classes/mcts.py'>